In [10]:
%reload_ext nb_black
import pandas as pd
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold
from xgboost import XGBClassifier

from sklearn.model_selection import GridSearchCV



import shap
from ipywidgets import interact_manual, fixed, widgets




%matplotlib inline

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Big Data Cup xG Model 

In [11]:
# important numbers for the hockey rink 
ICE_LENGTH = 200
ICE_WIDTH = 85
GOAL_X = ICE_LENGTH - 10
GOAL_Y = ICE_WIDTH / 2
D_ZONE = 75
O_ZONE = ICE_LENGTH - 75


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
womens = pd.read_csv('womens_data_cleaned.csv')
womens.head()

,game_date,home_team,away_team,period,clock,home_team_skaters,away_team_skaters,home_team_goals,away_team_goals,team,...,zone_2,zone_diff,start_distance_to_goal,end_distance_to_goal,diff_x,diff_y,distance_covered,angle_to_goal_start,angle_to_goal_end,diff_angle_to_goal
0,2018-02-11,Olympic (Women) - Canada,Olympic (Women) - Olympic Athletes from Russia,1,1900-01-01 00:20:00,5,5,0,0,Olympic (Women) - Canada,...,2.0,0.0,90.001389,90.001389,0.0,0.0,0.000000,180.000000,180.000000,0.000000
1,2018-02-11,Olympic (Women) - Canada,Olympic (Women) - Olympic Athletes from Russia,1,1900-01-01 00:19:59,5,5,0,0,Olympic (Women) - Canada,...,2.0,0.0,101.986519,101.986519,0.0,0.0,0.000000,4.040816,4.040816,0.000000
2,2018-02-11,Olympic (Women) - Canada,Olympic (Women) - Olympic Athletes from Russia,1,1900-01-01 00:19:59,5,5,0,0,Olympic (Women) - Canada,...,2.0,0.0,92.402651,92.402651,0.0,0.0,0.000000,2.181818,2.181818,0.000000
3,2018-02-11,Olympic (Women) - Canada,Olympic (Women) - Olympic Athletes from Russia,1,1900-01-01 00:19:59,5,5,0,0,Olympic (Women) - Canada,...,3.0,1.0,92.402651,46.970736,62.0,-80.0,101.212647,2.181818,0.530120,-1.651698
4,2018-02-11,Olympic (Women) - Canada,Olympic (Women) - Olympic Athletes from Russia,1,1900-01-01 00:19:53,5,5,0,0,Olympic (Women) - Canada,...,3.0,0.0,46.970736,46.970736,0.0,0.0,0.000000,0.530120,0.530120,0.000000


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
womens.columns

Index(['game_date', 'home_team', 'away_team', 'period', 'clock',
       'home_team_skaters', 'away_team_skaters', 'home_team_goals',
       'away_team_goals', 'team', 'player', 'event', 'x_coord', 'y_coord',
       'detail_1', 'detail_2', 'detail_3', 'detail_4', 'player_2', 'x_coord_2',
       'y_coord_2', 'game_id', 'is_home', 'is_shot', 'is_goal', 'event_id',
       'team_id', 'player_id', 'detail_1_code', 'detail_2_code',
       'detail_3_code', 'detail_4_code', 'goal_diff', 'seconds_remaining',
       'poss_status', 'strength_state', 'zone_1', 'zone_2', 'zone_diff',
       'start_distance_to_goal', 'end_distance_to_goal', 'diff_x', 'diff_y',
       'distance_covered', 'angle_to_goal_start', 'angle_to_goal_end',
       'diff_angle_to_goal'],
      dtype='object')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
shots=womens.loc[(womens['event']=='Shot') | (womens['event']=='Goal')]
shots_model=womens.loc[:,['is_goal','x_coord','y_coord','diff_x','diff_y','start_distance_to_goal','angle_to_goal_start','strength_state']]
shots_model['c_coord'] = abs(shots_model['y_coord']-GOAL_Y)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
shots_model.head(10)

,is_goal,x_coord,y_coord,diff_x,diff_y,start_distance_to_goal,angle_to_goal_start,strength_state,c_coord
0,0,100,42,0.0,0.0,90.001389,180.000000,0.0,0.5
1,0,91,67,0.0,0.0,101.986519,4.040816,0.0,24.5
2,0,106,81,0.0,0.0,92.402651,2.181818,0.0,38.5
3,0,106,81,62.0,-80.0,92.402651,2.181818,0.0,38.5
4,0,168,1,0.0,0.0,46.970736,0.530120,0.0,41.5
5,0,168,1,29.0,25.0,46.970736,0.530120,0.0,41.5
6,0,4,64,0.0,0.0,187.238484,8.651163,0.0,21.5
7,0,4,64,6.0,5.0,187.238484,8.651163,0.0,21.5
8,0,18,78,24.0,4.0,175.625311,4.845070,0.0,35.5
9,0,58,82,46.0,-74.0,137.783344,3.341772,0.0,39.5


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
shots_model.count()

is_goal                   50884
x_coord                   50884
y_coord                   50884
diff_x                    50884
diff_y                    50884
start_distance_to_goal    50884
angle_to_goal_start       50884
strength_state            50884
c_coord                   50884
dtype: int64

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [17]:
features = ['x_coord','y_coord','start_distance_to_goal','angle_to_goal_start','strength_state']
labels = ['is_goal']


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [18]:
X = shots_model[features]
y = shots_model[labels]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [19]:
parameters = {
    'nthread': [4],
    'objective': ['binary:logistic'],
    'max_depth': [3, 4, 5, 6],
    'learning_rate': [0.01],
    'n_estimators': [100, 500, 1000],
    'seed': [42]
    }

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [20]:
df_xg_model = pd.DataFrame()
kf = KFold(10, shuffle=True)

for train_idx, test_idx in kf.split(shots_model):
    train_data = shots_model.iloc[train_idx].copy()
    test_data = shots_model.iloc[test_idx].copy()

    classifier = XGBClassifier()
    classifier = GridSearchCV(classifier, parameters, scoring='roc_auc', verbose=2)
    classifier.fit(
        train_data[features],
        train_data[labels]
    )
    dfs_predictions = {}
    y_pred = classifier.predict_proba(test_data[features])
    dfs_predictions[labels[0]] = pd.Series(y_pred[:,1], index=test_data.index)
    df_predictions = pd.concat(dfs_predictions, axis=1)
    df_xg_model = df_xg_model.append(df_predictions)
    


arning_rate=0.01, max_depth=3, n_estimators=1000, nthread=4, objective=binary:logistic, seed=42; total time=   2.9s
[21:12:21] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV] END learning_rate=0.01, max_depth=3, n_estimators=1000, nthread=4, objective=binary:logistic, seed=42; total time=   2.9s
[21:12:24] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV] END learning_rate=0.01, max_depth=3, n_estimators=1000, nthread=4, objective=binary:logistic, seed=42; total time=   2.9s
[21:12:27] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.c

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [22]:
shots_model['is_goal'].sum()

132

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [23]:
df_xg_model.sum()

is_goal    133.802536
dtype: float32

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [24]:
df_xg_model.describe()

,is_goal
count,50884.000000
mean,0.002630
std,0.012589
min,0.000048
25%,0.000060
50%,0.000082
75%,0.000548
max,0.408073


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [25]:
df_xg_model.sort_values(by=['is_goal'])

,is_goal
28580,0.000048
11691,0.000048
46821,0.000048
33951,0.000048
18633,0.000048
...,...
3138,0.334992
18551,0.359098
50694,0.400514
10296,0.405438


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [26]:
shots_model.loc[18552]

is_goal                     1.000000
x_coord                   188.000000
y_coord                    43.000000
diff_x                      2.000000
diff_y                     -0.500000
start_distance_to_goal      2.061553
angle_to_goal_start         4.000000
strength_state              1.000000
c_coord                     0.500000
Name: 18552, dtype: float64

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [264]:
womens.loc[18552]

game_date                                      2019-04-06
home_team                 Olympic (Women) - United States
away_team                        Olympic (Women) - Canada
period                                                  1
clock                                 1900-01-01 00:13:34
home_team_skaters                                       4
away_team_skaters                                       5
home_team_goals                                         1
away_team_goals                                         0
team                             Olympic (Women) - Canada
player                                        Sarah Nurse
event                                                Goal
x_coord                                               188
y_coord                                                43
detail_1                                         Snapshot
detail_2                                           On Net
detail_3                                                f
detail_4      

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [27]:
abs(df_xg_model.sum() - shots_model['is_goal'].sum())

is_goal    1.802536
dtype: float32

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [29]:
abs(df_xg_model.loc[womens.loc[(womens['event']=='Shot') | (womens['event']=='Goal')].index].sum() - shots_model['is_goal'].sum())

is_goal    60.677193
dtype: float32

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [30]:
df_xg_model.describe()

,is_goal
count,50884.000000
mean,0.002630
std,0.012589
min,0.000048
25%,0.000060
50%,0.000082
75%,0.000548
max,0.408073


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [36]:
womens['non_shot_xg'] = df_xg_model['is_goal']

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [37]:
womens['non_shot_xg']

0        0.000074
1        0.000069
2        0.000056
3        0.000056
4        0.000355
           ...   
50879    0.000098
50880    0.000084
50881    0.000064
50882    0.000255
50883    0.000129
Name: non_shot_xg, Length: 50884, dtype: float32

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [41]:
df_rank_events = (womens[['event','non_shot_xg']]
.groupby(['event'])
.agg(non_shot_xg_count=('non_shot_xg','count'),
non_shot_xg_mean=('non_shot_xg','mean'),
non_shot_xg_sum=('non_shot_xg','sum'))
.sort_values('non_shot_xg_sum',ascending=False)
.reset_index()
)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [42]:
df_rank_events

,event,non_shot_xg_count,non_shot_xg_mean,non_shot_xg_sum
0,Shot,3524,0.018028,63.530735
1,Puck Recovery,15174,0.002229,33.827202
2,Play,14673,0.001038,15.234135
3,Goal,132,0.059031,7.792071
4,Incomplete Play,6111,0.001254,7.661183
5,Takeaway,2092,0.000887,1.856569
6,Penalty Taken,260,0.005496,1.428989
7,Zone Entry,3744,0.000314,1.174332
8,Faceoff Win,1629,0.000581,0.947020
9,Dump In/Out,3545,0.000099,0.350296


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>